# Vector extraction using Inception V3 pre-trained model
## Import necessary packages

In [1]:
from __future__ import division, print_function
from keras import backend as K
from keras.layers import Input
from keras.callbacks import ModelCheckpoint
from keras.layers.core import Activation, Dense, Dropout, Lambda
from keras.layers.merge import Concatenate
from keras.models import Model, load_model
from keras.utils import np_utils
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import itertools
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from keras.optimizers import Adam
from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50

from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
import itertools
import numpy as np
import os
import pandas as pd

from PIL.Image import LANCZOS
from PIL import Image
from PIL import ImageFile
Image.MAX_IMAGE_PIXELS = 1000000000                                                                                              
ImageFile.LOAD_TRUNCATED_IMAGES = True

Using TensorFlow backend.


In [2]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
dat = pd.read_csv('/home/nelssalminen/painters/data/train_info.csv')
IMG_LIST = dat['filename'].tolist()
IMG_DIR = '/home/nelssalminen/painters/data/train/'
OUTPUT_DIR = '/home/nelssalminen/painters/data/output/'

os.makedirs(OUTPUT_DIR, exist_ok=True)

print(len(IMG_LIST))

model = InceptionV3(weights='imagenet',include_top=False,pooling='avg')

79433
Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
#fvec = open(OUTPUT_DIR + 'inceptionv3-vectors.tsv', "w")
num_vecs = 0 
for image_ in IMG_LIST:
	img = image.load_img(IMG_DIR+image_, target_size=(224, 224))
	x = image.img_to_array(img)
	x = np.expand_dims(x, axis=0)
	x = preprocess_input(x)
	features = model.predict(x)[0]
	# Convert from numpy array to a list of values
	features_arr = np.char.mod('%f', features)

	if num_vecs % 100 == 0:
		print("{:d} vectors generated".format(num_vecs))

	image_vector = ",".join(["{:.5e}".format(v) for v in features.tolist()])
	fvec.write("{:s}\t{:s}\n".format(image_, image_vector))
	num_vecs += 1

# Vector classification
## Create image triples
We start this stage by grouping the images by painter. Once the images are organized, we enumerate through each
group of images per artist and randomly obtain a pair of a reference image and a *similar* image. Following this,
we randomly select an image from a different artist group, creating a pair of the same reference image, but now combined
with a dissimilar image.

In [4]:
def get_bottom_x_images(x):
    #create dictionary of all artists and paintings
    image_groups = {}
    for index, row in dat.iterrows():
        img_name = row['filename']
        group_name = row['artist']
        if group_name in image_groups:
            image_groups[group_name].append(img_name)
        else:
            image_groups[group_name] = [img_name]
    
    csv_top = []
    
    #create an arracy of artists and the number of paintings they've drawn
    for i in image_groups:
       csv_top.append((i,len(image_groups[i])))
    #sort the above array by ranking of number of paintings
    csv_top = sorted(csv_top,key=lambda l:l[1], reverse=True)
    
    #define csv_top as list of bottom x artists
    csv_top = csv_top[-x:]
        
    top_image_groups = {}
    for i in csv_top:
        top_artist = i[0]
        for artist, image in image_groups.items():
            if top_artist == artist:
                top_image_groups[artist] = image
    return top_image_groups

list_image = get_bottom_x_images(80)
excluded_artists = list(list_image.keys())
print(excluded_artists)
excluded_images = [item for sublist in list_image.values() for item in sublist]

['5ccd38b260edb40fc788b86761475dc3', '78e52c4888ac7356d3f9330c7f6d8efa', 'f241a8a2df78010687adc1b433248db2', '4f5f7ea6bfde0b20405cf17c3d080e8d', '24737e13e24bf959b89f88d35aae6f18', 'bf3d528ccc483dbae4421d5d4486dcf4', '4928068b969eb94d712513fb37775b69', '4bd6166a198cb687545a6e62f735458f', 'a921bba6adb1ef505aaaf2f7d993259c', 'b63534983e7d5389c3eb238a99a01f1f', '65972a3cdf808987b805b2f77b84f767', '58d8e97010e5340295564f9b26a91c9e', '5c4a0fe7dd40c408a6d094cb117b2ddf', 'c8792a07d9c3e3aaa94cf7c0b3bce17d', 'b3c8aac6b09a9462f18eaddcc8d8c8b6', '25b752932874a46bf760fbb4ba3ae154', 'a5f516b48d9be09d865a17e28f578664', 'c4a9bd575234cb4fd3ac9bb465801133', '7df32a5921612479ead3053cc3fef603', 'cc855b3d3c291ffba43e07934e8fb616', '59f6f564b33837275aebc04294fb23b9', 'f5a190b5e7c20a137c614799eebe9479', '6da85288541891498dd65e07d3b64371', '464fee66276dfef8e065a8d60d2e8b42', '6ec94ae04aa9443ab71f504e5e5af607', '5d9e88804f8a905749ce7ca7a86301df', 'fc1f3b2cf51749fec8a6496e02e26673', '79c5b2fac57715daff80d6ca92

In [5]:
def show_img(sid, img_file, img_title):
    plt.subplot(sid)
    plt.title(img_title)
    plt.xticks([])
    plt.yticks([])
    img = imresize(plt.imread(img_file), (512, 512))
    plt.imshow(img)

def get_triples(image_dir):
        image_groups = {}
        for index, row in dat.iterrows():
            img_name = row['filename']
            group_name = row['artist']
            if group_name not in excluded_artists:
                if group_name in image_groups:
                    image_groups[group_name].append(img_name)
                else:
                    image_groups[group_name] = [img_name]

        num_sims = 0
        image_triples = []
        group_list = sorted(list(image_groups.keys()))
        for i, g in enumerate(group_list):
                if g in excluded_artists:
                    print("Wait a minute here... hmmmm")
                if num_sims % 100 == 0:
                        print("Generated {:d} pos + {:d} neg = {:d} total image triples"
                                    .format(num_sims, num_sims, 2*num_sims))
                images_in_group = image_groups[g]
                sim_pairs_it = itertools.combinations(images_in_group, 2)
                # For each similar pair, generate a corresponding different pair
                for ref_image, sim_image in sim_pairs_it:
                    image_triples.append((ref_image, sim_image, 1))
                    num_sims += 1
                    while True:
                            j = np.random.randint(low=0, high=len(group_list), size=1)[0]
                            if j != i:
                                    break
                    dif_image_candidates = image_groups[group_list[j]]
                    k = np.random.randint(low=0, high=len(dif_image_candidates), size=1)[0]
                    dif_image = dif_image_candidates[k]
                    image_triples.append((ref_image, dif_image, 0))
#                     if num_sims % 10000 == 0:
#                         show_img(131, os.path.join(IMG_DIR, sim_image), "sim")
#                         show_img(132, os.path.join(IMG_DIR, ref_image), "ref")
#                         show_img(133, os.path.join(IMG_DIR, dif_image), "dif")
#                         plt.tight_layout()
#                         plt.show()


        print("Generated {:d} pos + {:d} neg = {:d} total image triples"
                    .format(num_sims, num_sims, 2*num_sims))
        return image_triples

Generate the image triples using the previously defined function.

In [6]:
triples = get_triples(IMG_DIR);

Generated 0 pos + 0 neg = 0 total image triples
Generated 873100 pos + 873100 neg = 1746200 total image triples
Generated 987100 pos + 987100 neg = 1974200 total image triples
Generated 2472900 pos + 2472900 neg = 4945800 total image triples
Generated 3064200 pos + 3064200 neg = 6128400 total image triples
Generated 3102000 pos + 3102000 neg = 6204000 total image triples
Generated 3753800 pos + 3753800 neg = 7507600 total image triples
Generated 5361500 pos + 5361500 neg = 10723000 total image triples
Generated 5766400 pos + 5766400 neg = 11532800 total image triples
Generated 5772145 pos + 5772145 neg = 11544290 total image triples


Set several configuration and utility variables, including loading the vector files.

In [7]:
def load_vectors(vector_file, excl):
        vec_dict = {}
        fvec = open(vector_file, "r")
        for line in fvec:
                image_name, image_vec = line.strip().split("\t")
                if image_name not in excl:
                    vec = np.array([float(v) for v in image_vec.split(",")])
                    vec_dict[image_name] = vec
        fvec.close()
        return vec_dict

BATCH_SIZE = 256
NUM_EPOCHS = 35

DATA_CONTAINER = '/home/nelssalminen/painters/data/scratch/'
os.makedirs(DATA_CONTAINER, exist_ok=True)

VECTORIZERS = ["InceptionV3"]
MERGE_MODES = ["Concat", "Euclidean"]

scores = np.zeros((len(VECTORIZERS), len(MERGE_MODES)))

VECTOR_SIZE = 2048
VECTOR_FILE = os.path.join(OUTPUT_DIR, "inceptionv3-vectors.tsv")

vec_dict = load_vectors(VECTOR_FILE, excluded_images)

Split the triples for training, validation and testing based on given ratios.

In [8]:
def train_test_split(triples, splits):
        assert sum(splits) == 1.0
        split_pts = np.cumsum(np.array([0.] + splits))
        indices = np.random.permutation(np.arange(len(triples)))
        shuffled_triples = [triples[i] for i in indices]
        data_splits = []
        for sid in range(len(splits)):
                start = int(split_pts[sid] * len(triples))
                end = int(split_pts[sid + 1] * len(triples))
                data_splits.append(shuffled_triples[start:end])
        return data_splits

train_triples, val_triples, test_triples = train_test_split(triples, splits=[0.8, 0.1, 0.1])
print("Training set size: " + str(len(train_triples)), "Validation set size:" + str(len(val_triples)), "Test set size:" + str(len(test_triples)))

Training set size: 9235432 Validation set size:1154429 Test set size:1154429


*TODO: Add explanation*

In [9]:
def batch_to_vectors(batch, vec_size, vec_dict):
    X1 = np.zeros((len(batch), vec_size))
    X2 = np.zeros((len(batch), vec_size))
    Y = np.zeros((len(batch), 2))
    for tid in range(len(batch)):
        X1[tid] = vec_dict[batch[tid][0]]
        X2[tid] = vec_dict[batch[tid][1]]
        Y[tid] = [1, 0] if batch[tid][2] == 0 else [0, 1]
    return ([X1, X2], Y)


def data_generator(triples, vec_size, vec_dict, batch_size=32):
    while True:
        # shuffle once per batch
        indices = np.random.permutation(np.arange(len(triples)))
        num_batches = len(triples) // batch_size
        for bid in range(num_batches):
            batch_indices = indices[bid * batch_size: (bid + 1) * batch_size]
            batch = [triples[i] for i in batch_indices]
            yield batch_to_vectors(batch, vec_size, vec_dict)
            
train_gen = data_generator(train_triples, VECTOR_SIZE, vec_dict, BATCH_SIZE)
val_gen = data_generator(val_triples, VECTOR_SIZE, vec_dict, BATCH_SIZE)

In [10]:
input_1 = Input(shape=(VECTOR_SIZE,))
input_2 = Input(shape=(VECTOR_SIZE,))
merged = Concatenate(axis=-1)([input_1, input_2])

Define the 10-layer Siamese CNN

In [11]:
fc1 = Dense(2048, kernel_initializer="glorot_uniform")(merged)
fc1 = Dropout(0.2)(fc1)
fc1 = Activation("relu")(fc1)

fc2 = Dense(2048, kernel_initializer="glorot_uniform")(fc1)
fc2 = Dropout(0.2)(fc2)
fc2 = Activation("relu")(fc2)

fc3 = Dense(2048, kernel_initializer="glorot_uniform")(fc2)
fc3 = Dropout(0.2)(fc3)
fc3 = Activation("relu")(fc3)

fc8 = Dense(1024, kernel_initializer="glorot_uniform")(fc3)
fc8 = Dropout(0.2)(fc8)
fc8 = Activation("relu")(fc8)

fc9 = Dense(1024, kernel_initializer="glorot_uniform")(fc8)
fc9 = Dropout(0.2)(fc9)
fc9 = Activation("relu")(fc9)

fc11 = Dense(1024, kernel_initializer="glorot_uniform")(fc9)
fc11 = Dropout(0.2)(fc11)
fc11 = Activation("relu")(fc11)

fc12 = Dense(512, kernel_initializer="glorot_uniform")(fc11)
fc12 = Dropout(0.2)(fc12)
fc12 = Activation("relu")(fc12)

fc13 = Dense(512, kernel_initializer="glorot_uniform")(fc12)
fc13 = Dropout(0.2)(fc13)
fc13 = Activation("relu")(fc13)

fc14 = Dense(128, kernel_initializer="glorot_uniform")(fc13)
fc14 = Dropout(0.2)(fc14)
fc14 = Activation("relu")(fc14)

fc15 = Dense(128, kernel_initializer="glorot_uniform")(fc14)
fc15 = Dropout(0.2)(fc15)
fc15 = Activation("relu")(fc15)

pred = Dense(2, kernel_initializer="glorot_uniform")(fc15)
pred = Activation("softmax")(pred)

model = Model(inputs=[input_1, input_2], outputs=pred)
adam = Adam(lr=.00001)
model.compile(optimizer=adam, loss="categorical_crossentropy", metrics=["accuracy"])

model.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
concatenate_3 (Concatenate)     (None, 4096)         0           input_2[0][0]                    
                                                                 input_3[0][0]                    
_____________________________________________________________________________________________

Define functions for model evaluation

In [12]:
def evaluate_model(model_file, test_gen):
        model_name = os.path.basename(model_file)
        model = load_model(model_file)
        print("=== Evaluating model: {:s} ===".format(model_name))
        ytrue, ypred = [], []
        num_test_steps = len(test_triples) // BATCH_SIZE
        for i in range(num_test_steps):
                (X1, X2), Y = next(test_gen)
                Y_ = model.predict([X1, X2])
                ytrue.extend(np.argmax(Y, axis=1).tolist())
                ypred.extend(np.argmax(Y_, axis=1).tolist())
        accuracy = accuracy_score(ytrue, ypred)
        print("\nAccuracy: {:.3f}".format(accuracy))
        print("\nConfusion Matrix")
        print(confusion_matrix(ytrue, ypred))
        print("\nClassification Report")
        print(classification_report(ytrue, ypred))
        return accuracy
    
def get_model_file(data_dir, vector_name, merge_mode, borf):
        return os.path.join(data_dir, "models", "{:s}-{:s}-{:s}.h5"
                                                .format(vector_name, merge_mode, borf))

In [13]:
best_model_name = get_model_file(DATA_CONTAINER, "inceptionv3", "cat", "best")
checkpoint = ModelCheckpoint(best_model_name, save_best_only=True)
train_steps_per_epoch = len(train_triples) // BATCH_SIZE
val_steps_per_epoch = len(val_triples) // BATCH_SIZE
history = model.fit_generator(train_gen, steps_per_epoch=train_steps_per_epoch, 
                              epochs=NUM_EPOCHS, 
                              validation_data=val_gen, validation_steps=val_steps_per_epoch,
                              callbacks=[checkpoint])

final_model_name = get_model_file(DATA_CONTAINER, "inceptionv3", "cat", "final")
model.save(final_model_name)
test_gen = data_generator(test_triples, VECTOR_SIZE, vec_dict, BATCH_SIZE)
final_accuracy = evaluate_model(final_model_name, test_gen)

test_gen = data_generator(test_triples, VECTOR_SIZE, vec_dict, BATCH_SIZE)
best_accuracy = evaluate_model(best_model_name, test_gen)

scores[0, 0] = best_accuracy if best_accuracy > final_accuracy else final_accuracy

Instructions for updating:
Use tf.cast instead.
Epoch 1/35
36075/36075 [==============================] - 735s 20ms/step - loss: 0.6379 - acc: 0.6245 - val_loss: 0.5880 - val_acc: 0.6909
Epoch 2/35
36075/36075 [==============================] - 742s 21ms/step - loss: 0.5524 - acc: 0.7102 - val_loss: 0.5202 - val_acc: 0.7529
Epoch 3/35
36075/36075 [==============================] - 742s 21ms/step - loss: 0.4911 - acc: 0.7568 - val_loss: 0.4686 - val_acc: 0.7911
Epoch 4/35
36075/36075 [==============================] - 744s 21ms/step - loss: 0.4462 - acc: 0.7881 - val_loss: 0.4322 - val_acc: 0.8167
Epoch 5/35
36075/36075 [==============================] - 742s 21ms/step - loss: 0.4121 - acc: 0.8104 - val_loss: 0.3972 - val_acc: 0.8334
Epoch 6/35
36075/36075 [==============================] - 740s 21ms/step - loss: 0.3860 - acc: 0.8268 - val_loss: 0.3816 - val_acc: 0.8451
Epoch 7/35
36075/36075 [==============================] - 744s 21ms/step - loss: 0.3655 - acc: 0.8391 - val_loss: 0.36